# 基于PBC数据集的循环DSM


纵向 PBC 数据集来源于 1974 年至 1984 年间由 Mayo Clinic 开展的针对原发性胆汁性肝硬化（PBC）的临床试验（详见 https://stat.ethz.ch/R-manual/R-devel/library/survival/html/pbc.html）。

在本笔记本中，我们将使用循环深度生存机（Recurrent Deep Survival Machines）对 PBC 数据进行生存预测。


### 加载 PBC 数据集

该软件包包含用于加载数据集的辅助函数。

X 表示特征（协变量）的 np.array，
T 是事件/删失时间，
E 是删失指示变量。


In [19]:
run compatibility_solution

In [20]:
# 从 auton_survival 库中导入 datasets 模块，该模块封装了常用的生存分析公开数据集
from auton_survival import datasets

# 调用 datasets.load_dataset 方法加载 PBC（原发性胆汁性胆管炎）数据集
# 参数 sequential=True 表示以时间序列格式返回，适用于需要按时间顺序建模的场景
# 返回值：
#   x: 患者随诊记录的多维特征（如实验室指标、年龄等），形状通常为 (n_samples, n_timesteps, n_features)
#   t: 每个样本对应的观测时间（单位：天），形状为 (n_samples,)
#   e: 事件指示符，1 表示发生终点事件（如死亡或肝移植），0 表示右删失，形状为 (n_samples,)
x, t, e = datasets.load_dataset('PBC', sequential=True)

# from dsm import datasets
# x, t, e = datasets.load_dataset('PBC', sequential = True)

In [21]:
x[1].shape

(9, 25)

In [22]:
t[1]

array([14.15233819, 13.6540357 , 13.15299529, 12.04961121,  9.2514511 ,
        8.26305991,  7.26645493,  6.26163618,  5.31978973])

In [23]:
e[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

### 计算用于评估 RDSM 性能的时间点

生存预测是在特定的时间点发出的。在此，我们将按照生存分析的标准做法，在第 25、50 和 75 个事件时间分位数处评估 RDSM 的预测性能。


In [24]:
import numpy as np                      # 导入 NumPy 库，用于数值计算
horizons = [0.25, 0.5, 0.75]            # 定义分位点列表：25%、50%、75% 分位
times = np.quantile(                    # 计算指定分位点的数值
    [t_[-1] for t_, e_ in zip(t, e)    # 列表推导：并行遍历 t 与 e，取 t 的最后一个元素
     if e_[-1] == 1],                   # 但仅保留 e 的最后一个元素为 1 的样本
    horizons                            # 对这些保留的 t 的末尾值计算 horizons 分位
).tolist()                              # 将结果数组转成 Python 列表并赋值给 times

### 将数据拆分为训练集、测试集和验证集
我们将在70%的数据上训练RDSM，使用10%的验证集进行模型选择，并在剩余的20%的测试集上报告性能。

In [25]:
# 获取样本总数，用于后续划分训练集、验证集和测试集
n = len(x)

# 计算训练集大小：取总样本数的70%，向下取整
tr_size = int(n*0.70)
# 计算验证集大小：取总样本数的10%，向下取整
vl_size = int(n*0.10)
# 计算测试集大小：取总样本数的20%，向下取整
te_size = int(n*0.20)

# 将特征x按顺序切分为训练集、测试集和验证集，并转换为object类型的numpy数组
# 训练集：前tr_size个样本
# 测试集：最后te_size个样本
# 验证集：紧接训练集后的vl_size个样本
x_train, x_test, x_val = np.array(x[:tr_size], dtype = object), np.array(x[-te_size:], dtype = object), np.array(x[tr_size:tr_size+vl_size], dtype = object)

# 同理，将标签t按相同顺序切分并转换为object类型的numpy数组
t_train, t_test, t_val = np.array(t[:tr_size], dtype = object), np.array(t[-te_size:], dtype = object), np.array(t[tr_size:tr_size+vl_size], dtype = object)

# 同理，将额外信息e按相同顺序切分并转换为object类型的numpy数组
e_train, e_test, e_val = np.array(e[:tr_size], dtype = object), np.array(e[-te_size:], dtype = object), np.array(e[tr_size:tr_size+vl_size], dtype = object)

### 设置参数网格

我们来设置参数网格以调优超参数。我们将调优底层生存分布的数量（$K$）、分布选择（Log-Normal 或 Weibull）、Adam 优化器的学习率（$1\times10^{-3}$ 到 $1\times10^{-4}$）、每层隐藏节点数（50、100 和 2）、层数（3、2 和 1）以及循环单元类型（LSTM、GRU、RNN）。


In [26]:
from sklearn.model_selection import ParameterGrid

In [27]:
# 定义超参数搜索网格：键为待调参数名，值为候选取值列表
param_grid = {'k' : [3, 4, 6],               # k：聚类或近邻数，候选 3/4/6
              'distribution' : ['LogNormal', 'Weibull'],  # distribution：数据分布假设，候选对数正态/威布尔
              'learning_rate' : [1e-4, 1e-3],             # learning_rate：优化器学习率，候选 0.0001/0.001
              'hidden': [50, 100],            # hidden：隐藏层单元数，候选 50/100
              'layers': [3, 2, 1],            # layers：网络层数，候选 3/2/1
              'typ': ['LSTM', 'GRU', 'RNN'],  # typ：循环网络类型，候选 LSTM/GRU/普通RNN
             }

# 使用 sklearn.model_selection 的 ParameterGrid 生成所有参数组合
params = ParameterGrid(param_grid)
# 返回的 params 是一个可迭代对象，每次迭代给出一个字典，对应网格中的一种超参数组合


### Model Training and Selection

In [28]:
# from dsm import DeepRecurrentSurvivalMachines

from auton_survival.models.dsm import DeepRecurrentSurvivalMachines


In [29]:
# 创建一个空列表，用于存储所有训练好的模型及其对应的验证集负对数似然（NLL）
models = []

# 遍历超参数列表 params，每个 param 是一个字典，包含一组超参数配置
for param in params:
    # 根据当前超参数配置实例化 DeepRecurrentSurvivalMachines 模型
    # k: 隐变量维度；distribution: 分布类型；hidden: 隐藏单元数；typ: 模型类型；layers: RNN 层数
    model = DeepRecurrentSurvivalMachines(k = param['k'],
                                 distribution = param['distribution'],
                                 hidden = param['hidden'], 
                                 typ = param['typ'],
                                 layers = param['layers'])
    
    # 使用训练数据 (x_train, t_train, e_train) 训练模型
    # iters=1 表示仅迭代一次（快速训练，用于网格搜索）；learning_rate 从当前 param 中读取
    model.fit(x_train, t_train, e_train, iters = 1, learning_rate = param['learning_rate'])
    
    # 计算该模型在验证集上的负对数似然（NLL），作为评估指标；NLL 越小越好
    # 将 [NLL, 模型] 作为嵌套列表追加到 models 列表中，便于后续排序
    models.append([[model.compute_nll(x_val, t_val, e_val), model]])

# 根据 NLL 值从小到大排序，选出验证集上表现最好的模型
# min() 会依据每个元素的第一个值（即 NLL）进行比较
best_model = min(models)

# 从 best_model 中提取出最优模型对象，供后续使用（如测试集评估或保存）
model = best_model[0][1]

100%|██████████| 1/1 [00:00<00:00, 32.68it/s]


### Inference

In [31]:
# 第1行：调用模型对象的 predict_risk 方法，输入测试特征 x_test 和待评估时间点 times，返回每个样本在对应时间点的风险（hazard）值或风险评分
# 用法：适用于生存分析中评估某时刻的瞬时风险，可用于后续排序或可视化风险曲线
out_risk = model.predict_risk(x_test, times)

# 第2行：调用模型对象的 predict_survival 方法，输入同样的 x_test 与 times，返回每个样本在对应时间点的生存概率（即 S(t) = P(T > t)）
# 用法：可直接绘制生存曲线，或计算中位生存时间、生存率差异等统计量
out_survival = model.predict_survival(x_test, times)

In [34]:
out_risk[:5]


array([[0.01161355, 0.02867651, 0.06792376],
       [0.0265857 , 0.0530141 , 0.1035099 ],
       [0.017294  , 0.03863475, 0.08298475],
       [0.02576799, 0.05145001, 0.10098833],
       [0.07299215, 0.11831772, 0.18721085]])

### 评估

我们在拼接的时间序列数据上评估 RDSM 的性能，包括其判别能力（时间依赖性一致性指数和累积动态 AUC）以及 Brier 评分。


In [35]:
from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc

In [ ]:
# 初始化两个空列表，分别用于存储不同时间点的 C-index（一致性指数）和 Brier Score（Brier 评分）
cis = []  # 存放每个时间点通过 IPCW 计算的 C-index
brs = []  # 存放 Brier Score 结果（注意：只 append 一次，后续用 brs[0] 索引）

# 将训练集的事件/时间二维列表展平为一维结构化数组，方便后续生存分析函数调用
# 每个元素是一个元组：(是否发生事件, 发生时间/删失时间)
et_train = np.array([(e_train[i][j], t_train[i][j]) for i in range(len(e_train)) for j in range(len(e_train[i]))],
                 dtype = [('e', bool), ('t', float)])
# 同样的处理应用于测试集
et_test = np.array([(e_test[i][j], t_test[i][j]) for i in range(len(e_test)) for j in range(len(e_test[i]))],
                 dtype = [('e', bool), ('t', float)])
# 同样的处理应用于验证集（虽然后续未使用，但保持一致性）
et_val = np.array([(e_val[i][j], t_val[i][j]) for i in range(len(e_val)) for j in range(len(e_val[i]))],
                 dtype = [('e', bool), ('t', float)])

# 遍历所有待评估的时间点，计算 IPCW 调整后的 C-index，并收集结果
for i, _ in enumerate(times):
    cis.append(concordance_index_ipcw(et_train, et_test, out_risk[:, i], times[i])[0])
# 计算所有时间点的 Brier Score，返回元组中索引 [1] 为各时间点评分，一次性 append 到 brs
brs.append(brier_score(et_train, et_test, out_survival, times)[1])

# 初始化空列表，用于存储每个时间点对应的累积动态 AUC
roc_auc = []
for i, _ in enumerate(times):
    roc_auc.append(cumulative_dynamic_auc(et_train, et_test, out_risk[:, i], times[i])[0])

# 遍历 horizons（通常为时间分位点），打印对应指标
for horizon in enumerate(horizons):
    print(f"For {horizon[1]} quantile,")  # horizon[1] 为分位数值
    print("TD Concordance Index:", cis[horizon[0]])  # 对应时间点的 C-index
    print("Brier Score:", brs[0][horizon[0]])  # brs[0] 为所有时间点 Brier 评分列表
    print("ROC AUC ", roc_auc[horizon[0]][0], "\n")  # roc_auc[horizon[0]] 为元组，取 [0] 得 AUC 值


For 0.25 quantile,
TD Concordance Index: 0.9015748031496064
Brier Score: 0.004328819764850914
ROC AUC  0.9051383399209487 

For 0.5 quantile,
TD Concordance Index: 0.9471684503736253
Brier Score: 0.012148739865693587
ROC AUC  0.9555471011846373 

For 0.75 quantile,
TD Concordance Index: 0.8892376898017548
Brier Score: 0.0286592619508703
ROC AUC  0.8959676636108452 

